In [ ]:
import gc
import re
import math
import pickle
import joblib
import warnings

import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier

warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)

In [ ]:
import os
import random as rn
import numpy as np


def set_seed(seed=0):
    os.environ["PYTHONHASHSEED"] = str(seed)

    rn.seed(seed)
    np.random.seed(seed)

In [ ]:
from sklearn.metrics import log_loss


def score(Y, Y_pred):
    _, n_classes = Y.shape

    losses = []

    for j in range(n_classes):
        loss = log_loss(Y.iloc[:, j], Y_pred.iloc[:, j], labels=[0, 1])

        losses.append(loss)

    return np.mean(losses)

In [ ]:
import sys
sys.path.append('../input/iterativestratification')

import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection._split import _BaseKFold


class MultilabelGroupStratifiedKFold(_BaseKFold):
    def __init__(self, n_splits=5, random_state=None, shuffle=False):
        super().__init__(n_splits=n_splits, random_state=random_state, shuffle=shuffle)

    def _iter_test_indices(self, X=None, y=None, groups=None):
        cv = MultilabelStratifiedKFold(
            n_splits=self.n_splits,
            random_state=self.random_state,
            shuffle=self.shuffle,
        )

        value_counts = groups.value_counts()
        regular_index = value_counts.loc[
            (value_counts == 6) | (value_counts == 12) | (value_counts == 18)
        ].index.sort_values()
        irregular_index = value_counts.loc[
            (value_counts != 6) & (value_counts != 12) & (value_counts != 18)
        ].index.sort_values()

        group_to_fold = {}
        tmp = Y.groupby(groups).mean().loc[regular_index]

        for fold, (_, test) in enumerate(cv.split(tmp, tmp)):
            group_to_fold.update({group: fold for group in tmp.index[test]})

        sample_to_fold = {}
        tmp = Y.loc[groups.isin(irregular_index)]

        for fold, (_, test) in enumerate(cv.split(tmp, tmp)):
            sample_to_fold.update({sample: fold for sample in tmp.index[test]})

        folds = groups.map(group_to_fold)
        is_na = folds.isna()
        folds[is_na] = folds[is_na].index.map(sample_to_fold).values

        for i in range(self.n_splits):
            yield np.where(folds == i)[0]

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin


class ClippedFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, copy=True, high=0.99, low=0.01):
        self.copy = copy
        self.high = high
        self.low = low

    def fit(self, X, y=None):
        self.data_max_ = X.quantile(q=self.high)
        self.data_min_ = X.quantile(q=self.low)

        return self

    def transform(self, X):
        if self.copy:
            X = X.copy()

        X.clip(self.data_min_, self.data_max_, axis=1, inplace=True)

        return X

In [ ]:
import pandas as pd


def compute_row_statistics(X, prefix=""):
    Xt = pd.DataFrame()

    for agg_func in [
        # "min",
        # "max",
        "mean",
        "std",
        "kurtosis",
        "skew",
    ]:
        Xt[f"{prefix}{agg_func}"] = X.agg(agg_func, axis=1)

    return Xt

In [ ]:
dtype = {"cp_type": "category", "cp_dose": "category"}
index_col = "sig_id"

train_features = pd.read_csv(
    "../input/lish-moa/train_features.csv", dtype=dtype, index_col=index_col
)
X = train_features.select_dtypes("number")
Y_nonscored = pd.read_csv(
    "../input/lish-moa/train_targets_nonscored.csv", index_col=index_col
)
Y = pd.read_csv("../input/lish-moa/train_targets_scored.csv", index_col=index_col)
groups = pd.read_csv(
    "../input/lish-moa/train_drug.csv", index_col=index_col, squeeze=True
)

columns = Y.columns

In [ ]:
clipped_features = ClippedFeatures()
X = clipped_features.fit_transform(X)

with open("clipped_features.pkl", "wb") as f:
    pickle.dump(clipped_features, f)

# アンサンブルのため統計値, nonscoredは入れない 
#c_prefix = "c-"
#g_prefix = "g-"
#c_columns = X.columns.str.startswith(c_prefix)
#g_columns = X.columns.str.startswith(g_prefix)
#X_c = compute_row_statistics(X.loc[:, c_columns], prefix=c_prefix)
#X_g = compute_row_statistics(X.loc[:, g_columns], prefix=g_prefix)
#X = pd.concat([X, X_c, X_g], axis=1)
#
#Y_nonscored = Y_nonscored.loc[:, Y_nonscored.sum(axis=0) > 0]
#Y = pd.concat([Y, Y_nonscored], axis=1)

In [ ]:
is_drug_cv = True
n_splits = 5
n_seeds = 5
# LBS = 0.0008  # ラベルスムージングは全然効かないからやめる
LBS = 0.0

params = {
    "objective": "binary",
    "learning_rate": 0.1,
    "num_leaves": 3,
    "max_depth": 2, 
    "min_data_in_leaf": 1367, 
    "feature_fraction": 0.65, 
    "lambda_l1": 3.6056855520901, 
    "lambda_l2": 0.2777268104182044
}

#DEBUG = True
DEBUG = False
if DEBUG:
    columns = [
        "atp-sensitive_potassium_channel_antagonist",  # 陽性ラベル1個だけ
        "erbb2_inhibitor",  # 陽性ラベル1個だけ
        "antiarrhythmic",  # 陽性ラベル6個だけ
        "aldehyde_dehydrogenase_inhibitor",  # 陽性ラベル7個だけ
        "lipase_inhibitor",  # 陽性ラベル12個だけ
        "sphingosine_receptor_agonist",  # 陽性ラベル25個だけ
        "igf-1_inhibitor",  # 陽性ラベル37個だけ
        "potassium_channel_activator",  # 陽性ラベル55個だけ
        "potassium_channel_antagonist",  # 陽性ラベル98個だけ
        "dopamine_receptor_agonist",  # 陽性ラベル121個だけ
        "nfkb_inhibitor",  # 陽性ラベル832個
        "cyclooxygenase_inhibitor",  # 陽性ラベル435個
        "dna_inhibitor",  # 陽性ラベル402個
        "glutamate_receptor_antagonist",  # 陽性ラベル367個
        "tubulin_inhibitor",  # 陽性ラベル316個
        "pdgfr_inhibitor",  # 陽性ラベル297個
        "calcium_channel_blocker",  # 陽性ラベル281個
        "flt3_inhibitor",  # 陽性ラベル279個
        "progesterone_receptor_agonist",  # 陽性ラベル119個
        "hdac_inhibitor",  # 陽性ラベル106個
    ]
    Y = Y[columns]

    #non_columns = [
    #    "abc_transporter_expression_enhancer",  # nonscored class
    #    "abl_inhibitor",  # nonscored class
    #]
    #Y_nonscored = Y_nonscored[non_columns]
    #
    #Y = pd.concat([Y, Y_nonscored], axis=1)
    
    params["n_estimators"] = 2
    n_seeds = 2
    n_splits = 2
    print(f"DEBUG: {DEBUG}")


In [ ]:
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

In [ ]:
%%time

counts = np.empty((n_seeds * n_splits, n_classes))

Y_pred = np.zeros((train_size, n_classes))
Y_pred = pd.DataFrame(Y_pred, columns=Y.columns, index=Y.index)

for i in range(n_seeds):
    set_seed(seed=i)
    
    if is_drug_cv:
        cv = MultilabelGroupStratifiedKFold(n_splits=n_splits, random_state=i, shuffle=True)
        cv_split = cv.split(X, Y, groups)
    else:
        MultilabelStratifiedKFold(n_splits=n_splits, random_state=i, shuffle=True)
        cv_split = cv.split(X, Y)

    order = rn.sample(range(Y.shape[1]), k=Y.shape[1])
    print(f"order: {order}")
        
    for j, (trn_idx, val_idx) in tqdm(enumerate(cv_split)):
        counts[i * n_splits + j] = Y.iloc[trn_idx].sum()
        
        X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
        Y_train, Y_val = Y.iloc[trn_idx], Y.iloc[val_idx]

        # Label Smoothing. https://www.kaggle.com/gogo827jz/self-stacking-groupcv-xgboost
        Y_train = Y_train * (1 - LBS) + 0.5 * LBS
        
        clf = ClassifierChain(LGBMClassifier(**params), order=order)
        clf.fit(X_train, Y_train)
        val_preds = clf.predict_proba(X_val) # list of preds per class
        
        Y_pred.iloc[val_idx] += val_preds / n_seeds
        
        joblib.dump(clf, f"model_seed_{i}_fold_{j}.jlb", compress=True)

Y_pred[train_features["cp_type"] == "ctl_vehicle"] = 0.0

with open("counts.pkl", "wb") as f:
    pickle.dump(counts, f)

with open("Y_pred.pkl", "wb") as f:
    pickle.dump(Y_pred[columns], f)

In [ ]:
score(Y[columns], Y_pred[columns])

# Platt Scaling
Train a Logistic Regression model to calibrate the results
- https://www.kaggle.com/gogo827jz/kernel-logistic-regression-one-for-206-targets

In [ ]:
# predict_probaでだしたY_predをロジスティク回帰で確率に補正する

counts = np.empty((n_classes))

X_new = Y_pred.values
Y_cali = Y_pred.copy()

for tar in tqdm(range(Y.shape[1])):
    
    targets = Y.values[:, tar]
    X_targets = X_new[:, tar]
    counts[tar] = targets.sum()
    
    print(tar, counts[tar])

    if targets.sum() >= n_splits:
        
        Y_cali[Y.columns[tar]] = np.zeros((Y_cali.shape[0], ))

        skf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)

        for n, (tr, te) in enumerate(skf.split(targets, targets)):
            x_tr, x_val = X_targets[tr].reshape(-1, 1), X_targets[te].reshape(-1, 1)
            y_tr, y_val = targets[tr], targets[te]

            model = LogisticRegression(penalty="none", max_iter=1000)
            model.fit(x_tr, y_tr)
            Y_cali[Y.columns[tar]].iloc[te] += model.predict_proba(x_val)[:, 1]
            
            joblib.dump(model, f"calibrate_model_target_{Y.columns[tar]}.jlb", compress=True)

with open("counts_calibrate.pkl", "wb") as f:
    pickle.dump(counts, f)

with open("Y_pred_calibrate.pkl", "wb") as f:
    pickle.dump(Y_cali[columns], f)
    

In [ ]:
score(Y[columns], Y_cali[columns])

# pkl check

In [ ]:
path = r"counts.pkl"
with open(path, 'rb') as f:
    counts = pickle.load(f)
counts

In [ ]:
path = r"counts_calibrate.pkl"
with open(path, 'rb') as f:
    counts = pickle.load(f)
counts

In [ ]:
path = r"Y_pred.pkl"
with open(path, 'rb') as f:
    Y_pred = pickle.load(f)
Y_pred

In [ ]:
path = r"Y_pred_calibrate.pkl"
with open(path, 'rb') as f:
    Y_pred = pickle.load(f)
Y_pred

# predict test

In [ ]:
import glob
import pathlib


test_features = pd.read_csv(
    "../input/lish-moa/test_features.csv", dtype=dtype, index_col=index_col,
)
X_test = test_features.select_dtypes("number")

with open("./clipped_features.pkl", "rb") as f:
    clipped_features = pickle.load(f)
X_test = clipped_features.transform(X_test)
# アンサンブルのために統計値, nonscoredは入れない
# X_c = compute_row_statistics(X_test.loc[:, c_columns], prefix=c_prefix)
# X_g = compute_row_statistics(X_test.loc[:, g_columns], prefix=g_prefix)
# X_test = pd.concat([X_test, X_c, X_g], axis=1)

# lgbで予測
Y_test_pred = np.zeros((X_test.shape[0], len(columns)))
Y_test_pred = pd.DataFrame(Y_test_pred, columns=columns, index=test_features.index)
for i in range(n_seeds):
    set_seed(seed=i)

    for j in range(n_splits):
        clf = joblib.load(f"model_seed_{i}_fold_{j}.jlb")
        Y_test_pred += clf.predict_proba(X_test)[:, : len(columns)] / (
            n_seeds * n_splits
        )

print(Y_test_pred.shape)
display(Y_test_pred)

# lgbの予測値補正
model_paths = glob.glob(f"./calibrate_model_target_*.jlb")
for model_path in model_paths:
    target = str(pathlib.Path(model_path).stem).replace("calibrate_model_target_", "")

    if target in columns:
        # print(target)
        model = joblib.load(model_path)
        X_targets = Y_test_pred.loc[:, target].values.reshape(-1, 1)
        Y_test_pred.loc[:, target] = model.predict_proba(X_targets)[:, 1]

print(Y_test_pred.shape)
display(Y_test_pred)

In [ ]:
import glob
import pathlib


with open("counts.pkl", "rb") as f:
    counts = pickle.load(f)

denominator = counts.sum(axis=0)

counts /= denominator
print(counts)


test_features = pd.read_csv(
    "../input/lish-moa/test_features.csv", dtype=dtype, index_col=index_col,
)
X_test = test_features.select_dtypes("number")

with open("./clipped_features.pkl", "rb") as f:
    clipped_features = pickle.load(f)
X_test = clipped_features.transform(X_test)
# アンサンブルのために統計値, nonscoredは入れない
# X_c = compute_row_statistics(X_test.loc[:, c_columns], prefix=c_prefix)
# X_g = compute_row_statistics(X_test.loc[:, g_columns], prefix=g_prefix)
# X_test = pd.concat([X_test, X_c, X_g], axis=1)

# lgbで予測
Y_test_pred = np.zeros((X_test.shape[0], len(columns)))
Y_test_pred = pd.DataFrame(Y_test_pred, columns=columns, index=test_features.index)
for i in range(n_seeds):
    set_seed(seed=i)

    for j in range(n_splits):
        clf = joblib.load(f"model_seed_{i}_fold_{j}.jlb")
        # マイナークラスの確率下げるためにtrain setのfoldごとの陽性ラベルの割合かける
        Y_test_pred += counts[i * n_splits + j] * clf.predict_proba(X_test)[:, : len(columns)]

print(Y_test_pred.shape)
display(Y_test_pred)

# lgbの予測値補正
model_paths = glob.glob(f"./calibrate_model_target_*.jlb")
for model_path in model_paths:
    target = str(pathlib.Path(model_path).stem).replace("calibrate_model_target_", "")

    if target in columns:
        # print(target)
        model = joblib.load(model_path)
        X_targets = Y_test_pred.loc[:, target].values.reshape(-1, 1)
        Y_test_pred.loc[:, target] = model.predict_proba(X_targets)[:, 1]

print(Y_test_pred.shape)
display(Y_test_pred)